In [1]:
%load_ext autoreload
from coffea import hist, util

import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import numpy as np
import uproot

from pprint import pprint
import matplotlib.pyplot as plt

In [2]:
%autoreload 2
import sys 
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [3]:
from ttgenv.Utils.plotting import plotWithRatio

In [4]:
class PhotonSelector(processor.ProcessorABC):
    def __init__(self):
        # In the initializer, any of the outputs you would like to produce are defined (ex. histograms)

        # Coffea histograms are defined in the same way as in the previous exercise
        # define a list of axes first
        #Declare axis
        dataset_axis = hist.Cat("dataset","Dataset")
        
        ## Define axis to keep track of photon category
        phoCategory_axis = hist.Bin("category", r"Photon Category", [1,2,3,4,5])
        phoCategory_axis.identifiers()[0].label = "Genuine Photon"    
        phoCategory_axis.identifiers()[1].label = "Misidentified Electron"    
        phoCategory_axis.identifiers()[2].label = "Hadronic Photon"    
        phoCategory_axis.identifiers()[3].label = "Hadronic Fake"    
        
        #Declare axis 
        m3_axis = hist.Bin("M3", r"$M_3$ [GeV]", 200, 0., 1000)
        mass_axis = hist.Bin("mass", r"$m_{\ell\gamma}$ [GeV]", 400, 0., 400)
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 200, 0., 1000)
        eta_axis = hist.Bin("eta", r"$\eta_{\gamma}$", 300, -2.5, 2.5)
        chIso_axis = hist.Bin("chIso", r"Charged Hadron Isolation", np.arange(-0.1,20.001,.05))
        
        relIso_axis=hist.Bin("relIso", "relIso", 200,-4.5,4.5)
        phi_axis = hist.Bin("phi","$\phi$", 64, -3.2, 3.2)

        
        #Define the accumulator object, a dictionary storing all of the histograms and counters 
        #that we will fill later in the process function
        self._accumulator = processor.dict_accumulator({
            'photon_pt': hist.Hist("Counts", dataset_axis, pt_axis),
            'photon_eta': hist.Hist("Counts", dataset_axis, eta_axis),
            'photon_phi': hist.Hist("Counts", dataset_axis, phi_axis),
            'photon_mass': hist.Hist("Counts", dataset_axis, mass_axis),
            
            
            'charge':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'cleanmask':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'cutBased':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'cutBased17Bitmap':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'eCorr':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'electronIdx':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'electronIdxG':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'electronVeto':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'energyErr':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'genPartFlav':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'genPartIdx':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'genPartIdxG':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'hoe':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'isScEtaEB':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'isScEtaEE':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'jetIdx':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'jetIdxG':processor.column_accumulator(np.ndarray(shape=(1,8))),
           # 'mass':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID17':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID17_WP80':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID17_WP90':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID_WP80':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'mvaID_WP90':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'pdgId':processor.column_accumulator(np.ndarray(shape=(1,8))),
            
            'pfRelIso03_all':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'pfRelIso03_chg':processor.column_accumulator(np.ndarray(shape=(1,8))),
            
            'pixelSeed':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'r9':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'seedGain':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'sieie':processor.column_accumulator(np.ndarray(shape=(1,8))),
            'vidNestedWPBitmap':processor.column_accumulator(np.ndarray(shape=(1,8))),
            
            
            
            'EventCount': processor.value_accumulator(int),

        }
        )

    @property
    def accumulator(self):
        return self._accumulator

    

    # The process method is where the heart of the analysis is.  
    # This is where all of the selections are done and the histograms get filled 
    #  (things you did in notebook cells before will be done here instead)
    def process(self, events):
        ### The process function is where most of the work happens. As we'll see below, this is
        ### where the main analysis work happens (object cuts, event selections, filling histograms). 
        
        output = self.accumulator.identity()
        output['EventCount'] = len(events)

        dataset = events.metadata['dataset']
        ######################
        ###Object Selection###
        ######################
        
        #Add Tight Muon Select
        muons=events.Muon
        muonSelectTight = ((muons.pt>30) &
                           (abs(muons.eta)<2.4) &
                           (muons.tightId) &
                           (muons.pfRelIso04_all < 0.15)
                          )
        #Add Jet Select
        jets=events.Jet
        jetSelectTight = ((jets.pt>30) &
                          (abs(jets.eta)<2.4) &
                          (jets.isTight)
                         )

        #Add b-tagged Jet Select
        btaggedJetSelect = (jetSelectTight &
                           (jets.btagDeepB>0.6321)
                          )
        
        #Add Electron Select
        electrons=events.Electron
        electronSelectTight = ((electrons.pt> 35) &
                           (abs(electrons.eta)<2.1)
                         )
        #Add Photon Select
        photon=events.Photon
        photonSelect= ((photon.pt>20) & 
                        (abs(photon.eta) < 1.4442) #&
                        #(photon.isScEtaEE | photon.isScEtaEB) &
                       # (photon.electronVeto) & 
                        #np.invert(photon.pixelSeed) 
                       )
        photonID= photon.cutBased >=2
        #Apply Selection
        tightMuons = muons[muonSelectTight]
        tightJets = jets[jetSelectTight]
        tightBJets = jets[btaggedJetSelect]
        photons = photon[photonSelect]
        tightPho =photon[(photonID & photonSelect)]
        tightEle = electrons[electronSelectTight]
        
        #####################
        ###Event Selection###
        #####################
        
        #Apply trigger and add event selection 
        trigger = events.HLT.IsoMu24 | events.HLT.IsoTkMu24
        
        one_muon=(ak.num(tightMuons)==1)
        four_jets=(ak.num(tightJets)>=4)
        btag_jet=(ak.num(tightBJets)>=1)
        zero_ele=(ak.num(tightEle)==0)
        one_photon=(ak.num(tightPho)==1)
        
        # Select events passing the trigger, with exactly one tight muon, one tight photon, no electrons,  ≥4 jets, and ≥ 1 b-tagged jets. 
        eventSelection = (trigger &
                          one_muon &
                          four_jets & 
                          btag_jet &
                          one_photon &
                         zero_ele)
        
        E=events[eventSelection]
        #####################
        ###Photon Category###
        #####################
        
        #Add later
        
        ##Fill Histograms
        
        '''
        
        '''
        output['photon_pt'].fill(dataset=dataset,pt=ak.flatten(photons.pt[:,:1][eventSelection])) 
        output['photon_eta'].fill(dataset=dataset,eta=ak.flatten(photons.eta[:,:1][eventSelection])) 
        output['photon_mass'].fill(dataset=dataset,mass=ak.flatten(photons.mass[:,:1][eventSelection])) 
        output['photon_phi'].fill(dataset=dataset,phi=ak.flatten(photons.phi[:,:1][eventSelection])) 
    
        charge=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.charge,8,clip=True))))
        cleanmask=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.cleanmask,8,clip=True))))
        cutBased=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.cutBased,8,clip=True))))
        cutBased17Bitmap=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.cutBased17Bitmap,8,clip=True))))
        eCorr=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.eCorr,8,clip=True))))
        electronIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.electronIdx,8,clip=True))))
        electronIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.electronIdxG,8,clip=True))))
        electronVeto=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.electronVeto,8,clip=True))))
        energyErr=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.energyErr,8,clip=True))))
        genPartFlav=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.genPartFlav,8,clip=True))))
        genPartIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.genPartIdx,8,clip=True))))
        genPartIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.genPartIdxG,8,clip=True))))
        hoe=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.hoe,8,clip=True))))
        isScEtaEB=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.isScEtaEB,8,clip=True))))
        isScEtaEE=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.isScEtaEE,8,clip=True))))
        jetIdx=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.jetIdx,8,clip=True))))
        jetIdxG=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.jetIdxG,8,clip=True))))
        #'mass':processor.column_accumulator(np.ndarray(shape=(1,8))),
        mvaID=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID,8,clip=True))))
        mvaID17=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID17,8,clip=True))))
        mvaID17_WP80=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID17_WP80,8,clip=True))))
        mvaID17_WP90=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID17_WP90,8,clip=True))))
        mvaID_WP80=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID_WP80,8,clip=True))))
        mvaID_WP90=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.mvaID_WP90,8,clip=True))))
        pdgId=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.pdgId,8,clip=True))))
            
        pfRelIso03_all=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.pfRelIso03_all,8,clip=True))))
        pfRelIso03_chg=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.pfRelIso03_chg,8,clip=True))))
            
        pixelSeed=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.pixelSeed,8,clip=True))))
        r9=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.r9,8,clip=True))))
        seedGain=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.seedGain,8,clip=True))))
        sieie=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.sieie,8,clip=True))))
        vidNestedWPBitmap=processor.column_accumulator(np.array(ak.to_numpy(ak.pad_none(E.Photon.vidNestedWPBitmap,8,clip=True)))) 
    
        output['charge']+=charge
        output['cleanmask']+=cleanmask
        output['cutBased']+=cutBased
        output['cutBased17Bitmap']+=cutBased17Bitmap
        output['eCorr']+=eCorr
        output['electronIdx']+=electronIdx
        output['electronIdxG']+=electronIdxG
        output['electronVeto']+=electronVeto
        output['energyErr']+=energyErr
        output['genPartFlav']+=genPartFlav
        output['genPartIdx']+=genPartIdx
        output['genPartIdxG']+=genPartIdxG
        output['hoe']+=hoe
        output['isScEtaEB']+=isScEtaEB
        output['isScEtaEE']+=isScEtaEE
        output['jetIdx']+=jetIdx
        output['jetIdxG']+=jetIdxG
        output['mvaID']+=mvaID
        output['mvaID17']+=mvaID17
        output['mvaID17_WP80']+=mvaID17_WP80
        output['mvaID17_WP90']+=mvaID17_WP90
        output['mvaID_WP80']+=mvaID_WP80
        output['mvaID_WP90']+=mvaID_WP90
        output['pdgId']+=pdgId
        output['pfRelIso03_all']+=pfRelIso03_all
        output['pfRelIso03_chg']+=pfRelIso03_chg
        output['pixelSeed']+=pixelSeed
        output['r9']+=r9
        output['seedGain']+=seedGain
        output['sieie']+=sieie
        output['vidNestedWPBitmap']+=vidNestedWPBitmap

        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [6]:
#Define files to run over
skimDir="root://cmseos.fnal.gov//store/user/lpctop/TTGamma_FullRun2/Skims_v6-2/2016/"

fileset = {"TTGamma":[f"{skimDir}/TTGamma_SingleLept_2016_skim.root"],
           "TTbar":[f"{skimDir}/TTbarPowheg_Semilept_2016_skim_1of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_2of10.root"],
           "WGamma":[f"{skimDir}/WGamma_2016_skim.root"],
           "Z+jets":[f'{skimDir}/DYjetsM50_ext2_2016_skim_1of10.root'],
           "W+3jets":[f"{skimDir}/W3jets_2016_skim.root"],
           "W+4jets":[f"{skimDir}/W4jets_2016_skim.root"],
          }

filesetData = {"DataMu":[f"{skimDir}/Data_SingleMu_b_2016_skim_1of10.root"],
              }

In [7]:
np.warnings.filterwarnings('ignore')

#the NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

#Run Coffea code using uproot
outputMC = processor.run_uproot_job(
    fileset,  #dictionary of datasets to run on, defined earlier in this cell
    "Events", #Name of the TTree you will be opening
    PhotonSelector(),  #Coffea processor you defined
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},  ## workers = 2, parallelize jobs, running 2 at once
    chunksize=1000000, #in each chunk, use 1 million events
#     maxchunks=3, #limit to using only 3 chunks for each dataset (useful for testing purposes)
)

Preprocessing:   0%|          | 0/7 [00:00<?, ?file/s]

Processing:   0%|          | 0/23 [00:00<?, ?chunk/s]

In [8]:
outputData = processor.run_uproot_job(
    filesetData,
    "Events",
    PhotonSelector(),
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4}, 
    chunksize=1000000,
)


Preprocessing:   0%|          | 0/1 [00:00<?, ?file/s]

Processing:   0%|          | 0/6 [00:00<?, ?chunk/s]

Exception: Failed processing file: root://cmseos.fnal.gov//store/user/lpctop/TTGamma_FullRun2/Skims_v6-2/2016//Data_SingleMu_b_2016_skim_1of10.root (963750-1927500)